In [1]:
import sqlite3 as sql
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import yaml
import subprocess
import os

import runMops
import MopsAnalysis
import MopsReader
import MopsPlotter
import MopsDatabase
from MopsTracker import MopsTracker
from MopsParameters import MopsParameters

% matplotlib inline

In [2]:
### In order for this all to work, you'll need to add the NEOSIM repo to your python path.
### You will also need the following:
# eigen, boost, gsl (for MOPS)
# sims_maf (for this code)

In [3]:
dataDrive = "/Volumes/DataCenter/"
dataLocation = "neosimData/colin/dia_sources_to_mops.dat"
subsampleLocation = "neosimData/colin_2016_06_06/"
runLocation = dataDrive + subsampleLocation

In [4]:
def readDetectionsIntoDataframe(detsFile, header=1):
    return pd.read_csv(detsFile, header=header, names=["diaId", "visitId", "ssmId", "ra", "dec", "mjd", "mag", "snr"], delim_whitespace=True)

In [5]:
dets_df = readDetectionsIntoDataframe(dataDrive + dataLocation)
dets_df

,diaId,visitId,ssmId,ra,dec,mjd,mag,snr
0,84768485325931030,197367,1,189.456717,-5.587153,56398.008951,-7.034141,5.0
1,84768485325931045,197367,1,189.466819,-5.536871,56398.008951,-7.405883,5.0
2,84768485325931046,197367,1,189.467385,-5.569577,56398.008951,-6.741032,5.0
3,84768485325931051,197367,1,189.471613,-5.554848,56398.008951,-7.583233,5.0
4,84768485325931063,197367,1,189.482410,-5.533385,56398.008951,-8.145744,5.0
5,84768485325931095,197367,1,189.504929,-5.570673,56398.008951,-6.760771,5.0
6,84768485325931098,197367,1,189.506098,-5.570450,56398.008951,-6.799934,5.0
7,84768485325931110,197367,1,189.516017,-5.593709,56398.008951,-6.992247,5.0
8,84768485325931233,197367,1,189.597998,-5.599326,56398.008951,-6.887967,5.0
9,84768485325931237,197367,1,189.599544,-5.603819,56398.008951,-7.341438,5.0


In [6]:
reduced_dets_df = dets_df.drop("diaId", 1)
reduced_dets_df

,visitId,ssmId,ra,dec,mjd,mag,snr
0,197367,1,189.456717,-5.587153,56398.008951,-7.034141,5.0
1,197367,1,189.466819,-5.536871,56398.008951,-7.405883,5.0
2,197367,1,189.467385,-5.569577,56398.008951,-6.741032,5.0
3,197367,1,189.471613,-5.554848,56398.008951,-7.583233,5.0
4,197367,1,189.482410,-5.533385,56398.008951,-8.145744,5.0
5,197367,1,189.504929,-5.570673,56398.008951,-6.760771,5.0
6,197367,1,189.506098,-5.570450,56398.008951,-6.799934,5.0
7,197367,1,189.516017,-5.593709,56398.008951,-6.992247,5.0
8,197367,1,189.597998,-5.599326,56398.008951,-6.887967,5.0
9,197367,1,189.599544,-5.603819,56398.008951,-7.341438,5.0


In [7]:
reduced_dets_df.to_csv(dataDrive + "neosimData/colin/dia_sources_to_mops_mod.dat", sep=" ", header=False)

In [8]:
nightly = dataDrive + "neosimData/colin/nightly/"

! rm -rf {nightly}
! mkdir {nightly}

In [9]:
call = ["python", os.getenv("MOPS_DIR") + "/bin/splitByNight.py", "-n", nightly, dataDrive + "neosimData/colin/dia_sources_to_mops_mod.dat"]
subprocess.call(call);

In [10]:
parameters = MopsParameters(velocity_max="2.0")
tracker = MopsTracker(runLocation)
tracker.getDetections(nightly)

------- MOPS Parameters --------
Current Parameter Values:

---- findTracklets ----
	Maximum velocity:                         2.0
	Minimum velocity:                         0.0
---- collapseTracklets ----
	Right Ascension tolerance:                0.002
	Declination tolerance:                    0.002
	Angular tolerance:                        5
	Velocity tolerance:                       0.05
	Method:                                   greedy
	Use RMS filter:                           True
	Maximum RMS:                              0.001
---- purifyTracklets ----
	Maximum RMS:                              0.001
---- removeSubsets (tracklets) ----
	Remove subsets:                           False
	Keep only longest:                        False
---- makeLinkTrackletsInput_byNight.py ----
	Window size:                              15
---- linkTracklets ----
	Detection error threshold:                0.0004
	Maximum right ascension acceleration:     0.02
	Maximum declination acceleration: 

In [11]:
parameters, tracker = runMops.runMops(parameters, tracker, linkTracklets=False, removeSubsetTracks=False, overwrite=True)

------- Run MOPS -------
Running LSST's Moving Object Pipeline

Overwrite triggered: clearing tracker...

Found 4 detection files in /Volumes/DataCenter/neosimData/colin/nightly/.

Overwrite triggered: deleting existing directory...

Saving parameters to /Volumes/DataCenter/neosimData/colin_2016_06_06/parameters.yaml

------- Run MOPS -------
Running findTracklets...
Completed running findTracklets.

Saving tracker to /Volumes/DataCenter/neosimData/colin_2016_06_06/tracker.yaml

------- Run MOPS -------
Running idsToIndices.py...
Completed running idsToIndices.py.

Saving tracker to /Volumes/DataCenter/neosimData/colin_2016_06_06/tracker.yaml

------- Run MOPS -------
Running collapseTracklets...
Completed running collapseTracklets.

------- Run MOPS -------
Running indicesToIds.py...
Completed running indicesToIds.py.

Saving tracker to /Volumes/DataCenter/neosimData/colin_2016_06_06/tracker.yaml

------- Run MOPS -------
Running purifyTracklets...
Completed running purifyTracklets.



In [12]:
parameters = MopsParameters.fromYaml(runLocation + "parameters.yaml")
tracker = MopsTracker.fromYaml(runLocation + "tracker.yaml")

Loading parameters from /Volumes/DataCenter/neosimData/colin_2016_06_06/parameters.yaml
Loading tracker from /Volumes/DataCenter/neosimData/colin_2016_06_06/tracker.yaml


In [13]:
results, objects_df = MopsAnalysis.analyze(parameters, tracker, tracklets=True, tracks=False, toDatabase=True, fullDetFile=dataDrive + "neosimData/colin/dia_sources_to_mops_mod.dat", overwrite=True)

Initializing new results object...
Creating DiaSources table...
Creating AllObjects table...
Creating FoundObjects view...
Creating MissedObjects view...
Creating AllTracklets table...
Creating TrackletMembers table...
Creating Tracklets view...
Creating CollapsedTracklets view...
Creating PurifiedTracklets view...
Creating FinalTracklets view...

Reading full detections file into dataframe...
Counting findable objects as tracklets...
Counting findable objects as tracks...
Building objects dataframe...
Updating objects dataframe...
Reading full detections file into database...

Starting tracklet analysis for 4 nights...

Starting tracklet analysis for night 56398 at Mon Jun  6 16:34:24 2016

- Writing results to /Volumes/DataCenter/neosimData/colin_2016_06_06/results/56398.results
- Checking file sizes...
- Reading input detections...
- Counting findable objects...
- Updating results object...
Saving results to /Volumes/DataCenter/neosimData/colin_2016_06_06/results/results.yaml
- Writ

In [21]:
objects_df

,objectId,numDetections,findableAsTracklet,findableAsTrack,numFalseTracklets,numTrueTracklets,numFalseCollapsedTracklets,numTrueCollapsedTracklets,numFalsePurifiedTracklets,numTruePurifiedTracklets,numFalseFinalTracklets,numTrueFinalTracklets,numFalseTracks,numTrueTracks,numFalseFinalTracks,numTrueFinalTracks
0,1,27248,True,True,0,19050,0,95,0,16,0,0,0,0,0,0


In [20]:
# Hmm, this table will probably not be useful for you. 
# Neither will a lot of the analysis code which calculates completeness and linking efficiency using objectId.
# However, the results database will have useful stuff for you. 
# Just a bit of disclaimer: 
# most of this code is very experimental and still in the very early stages of development. On top of that,
# its been put on the back-burner until we start the re-write of the MOPS software.

In [15]:
con = sql.connect(tracker.mainDatabase)

In [16]:
pd.read_sql("SELECT * FROM AllTracklets", con)

,trackletId,linkedObjectId,numLinkedObjects,numMembers,velocity,rms,night,createdBy,deletedBy
0,1,1,1,2,0.703317,1.013213e-14,56398.0,1,0
1,2,1,1,2,0.007177,1.152695e-13,56398.0,1,0
2,3,1,1,2,1.517911,1.022993e-14,56398.0,1,0
3,4,1,1,2,1.665857,8.270747e-14,56398.0,1,0
4,5,1,1,2,1.242958,6.463186e-14,56398.0,1,0
5,6,1,1,2,0.006462,4.675088e-14,56398.0,1,0
6,7,1,1,2,1.953564,2.841396e-14,56398.0,1,0
7,8,1,1,2,0.626960,4.674965e-14,56398.0,1,0
8,9,1,1,2,1.071506,4.569822e-14,56398.0,1,0
9,10,1,1,2,0.538435,1.007397e-14,56398.0,1,0


In [17]:
pd.read_sql("SELECT * FROM CollapsedTracklets", con)

,trackletId,linkedObjectId,numLinkedObjects,numMembers,velocity,rms,night,createdBy,deletedBy
0,7547,1,1,2,1.540227,8.087707e-14,56401.0,1,2
1,7559,1,1,2,1.774521,9.271197e-14,56401.0,1,2
2,7560,1,1,2,1.824286,4.540482e-14,56401.0,1,2
3,7729,1,1,2,0.264855,6.423866e-14,56401.0,1,2
4,7733,1,1,2,0.263204,9.273156e-14,56401.0,1,2
5,7746,1,1,2,1.718490,9.278675e-14,56401.0,1,2
6,7748,1,1,2,1.736854,4.536876e-14,56401.0,1,2
7,7796,1,1,2,0.351508,4.091207e-14,56401.0,1,2
8,7803,1,1,2,0.348587,9.271191e-14,56401.0,1,2
9,7806,1,1,2,0.295657,3.978967e-14,56401.0,1,2


In [18]:
pd.read_sql("SELECT * FROM PurifiedTracklets", con)

,trackletId,linkedObjectId,numLinkedObjects,numMembers,velocity,rms,night,createdBy,deletedBy
0,19146,1,1,2,1.541486,6.426636e-14,56401.0,3,0
1,19147,1,1,2,1.824286,4.540482e-14,56401.0,3,0
2,19148,1,1,2,1.922276,1.416914e-13,56401.0,3,0
3,19149,1,1,2,0.004247,9.267041e-14,56401.0,3,0
4,19150,1,1,2,1.325128,4.534530e-14,56401.0,3,0
5,19151,1,1,2,1.852919,9.265258e-14,56401.0,3,0
6,19152,1,1,2,1.791539,1.142971e-13,56401.0,3,0
7,19153,1,1,2,1.863874,9.260942e-14,56401.0,3,0
8,19154,1,1,2,0.006072,6.409695e-14,56401.0,3,0
9,19155,1,1,2,1.544500,9.254445e-14,56401.0,3,0


In [19]:
# No subset tracklets were found
pd.read_sql("SELECT * FROM finalTracklets", con)

,trackletId,linkedObjectId,numLinkedObjects,numMembers,velocity,rms,night,createdBy,deletedBy
